## The analytical task objective

Assume the scenario of the project:  
Zhang San is a bookstore owner. He plans to open several bookstores in Macao. After investigation, it is found that the people who love reading in Macao mainly live in the 50 buildings selected above. Zhang San should consider where the bookstore would be most conducive to the operation of the bookstore in Macao. Therefore, the number of bookstores and the location relationship between bookstores and buildings can effect the operation of the bookstores. At the same time, he has two trucks for incoming goods, and needs to design an optimal distribution route.

## The challenges of this project and the main solutions

This experiment solves these problems through clustering and path planning.  

**Clustering**  
Select the number of cluster centers according to the number of bookstores, and each bookstore serves the users of buildings belonging to the cluster. The project using Kmeans algorithm to divided 50 buildings into 5 categories. Therefore, the 5 cluster centers are the location of bookstores.  
  
**Path planning**   
*Intra-class*  
The location of each bookstore should consider the shortest path of such buildings. The TSP problem implemented by greedy algorithm is used for path planning. The buildings in each category will be formed into a shortest route.  

*Inter-class*  
This project entails transporting goods from the port of entry to each bookstore. It will be solved by ortools library. The output is the weight to be carried by each vehicle and the distribution routes.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandana
import pandas as pd
from IPython.display import display, clear_output
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'geopandas'

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

ModuleNotFoundError: No module named 'ortools'

## Analytical Task

### 1. Clustering
#### Find the locations of bookstores

In [3]:
def extract_schools_buildings(place='Macau'):
    ox.config(log_console=True, use_cache=True)
    tags = {'building': ['residential']}
    all_buildings = ox.geometries_from_place(place, tags=tags)
    # we only interested in building names and it's geometric information
    all_buildings = all_buildings[['name', 'geometry']]
    all_buildings = all_buildings[~(all_buildings['name'].isnull())]
    all_buildings = all_buildings[~(all_buildings['geometry'].isnull())]
    all_buildings['center'] = gpd.points_from_xy(x=all_buildings['geometry'].bounds[['minx', 'maxx']].mean(axis=1),
                                                 y=all_buildings['geometry'].bounds[['miny', 'maxy']].mean(axis=1))

    tags = {'amenity': ['school']}
    all_schools = ox.geometries_from_place(place, tags=tags)
    all_schools = all_schools[~(all_schools['name'].isnull())]
    all_schools = all_schools[~(all_schools['geometry'].isnull())]
    all_schools['center'] = gpd.points_from_xy(x=all_schools['geometry'].bounds[['minx', 'maxx']].mean(axis=1),
                                               y=all_schools['geometry'].bounds[['miny', 'maxy']].mean(axis=1))
    return all_schools, all_buildings


def generate_case(all_schools, all_buildings, n=20):
    school = all_schools.sample(1).iloc[0]
    node_school = ox.distance.nearest_nodes(G, X=school['center'].x, Y=school['center'].y)
    buildings = all_buildings.sample(1)
    while True:
        tmp = buildings.iloc[-1]
        node_building = ox.distance.nearest_nodes(G, X=tmp['center'].x, Y=tmp['center'].y)
        if not (nx.has_path(G, node_school, node_building) and nx.has_path(G, node_building, node_school)):
            buildings.drop(buildings.tail(1).index, inplace=True)
        if len(buildings) >= n:
            break
        buildings = pd.concat([buildings, all_buildings.sample(1)])

    return school, buildings


1.1 Data Download(select 50 buildings in Macau)

In [4]:
ox.config(log_console=True, use_cache=True)
G = ox.graph_from_place('Macau', network_type='drive')
all_schools, all_buildings = extract_schools_buildings('Macau')
school, buildings = generate_case(all_schools, all_buildings, 50)
print(buildings['name'])

NameError: name 'ox' is not defined

1.2 KMeans clustering

In [5]:
buildings_X = buildings['center'].x.values.reshape(-1, 1)
buildings_Y = buildings['center'].y.values.reshape(-1, 1)
buildings_C = np.hstack((buildings_X, buildings_Y))

n_clusters = 5
model = KMeans(n_clusters=n_clusters)
epochs = 10
for i in range(epochs):
    y_pred = model.fit_predict(buildings_C)
center = model.cluster_centers_
# 画图显示样本数据
plt.figure('Kmeans', facecolor='lightgray')
plt.title('Kmeans', fontsize=16)
plt.xlabel('X', fontsize=14)
plt.ylabel('Y', fontsize=14)
plt.tick_params(labelsize=10)
plt.scatter(buildings_C[:, 0], buildings_C[:, 1], s=80, c=y_pred, cmap='brg', label='Samples')
plt.scatter(center[:, 0], center[:, 1], s=20, c="#000000")
plt.legend()
plt.show()

NameError: name 'buildings' is not defined

From the scatter plot, it can be seen that the 50 buildings are divided into 5 categories,  
and the 5 small black dots refer to the clustering results, i.e. the addresses of the bookstores.

1.3 Add clustering results to geodataframe data

In [6]:
# Find the subscript of the point closest to the center of each cluster
closest_pt_idx = []
for iclust in range(model.n_clusters):
    # get all points assigned to each cluster:
    cluster_pts = buildings_C[model.labels_ == iclust]
    # get all indices of points assigned to this cluster:
    cluster_pts_indices = np.where(model.labels_ == iclust)[0]

    cluster_cen = model.cluster_centers_[iclust]
    min_idx = np.argmin([euclidean(buildings_C[idx], cluster_cen) for idx in cluster_pts_indices])
    # print(cluster_pts_indices[min_idx])
    closest_pt_idx.append(cluster_pts_indices[min_idx])

buildings['class'] = None
for i in range(len(y_pred)):
    buildings['class'][i] = y_pred[i]

buildings_list = []
cluster_list = []
for i in range(n_clusters):
    buildings_tmp = buildings[buildings['class'] == i]
    buildings_list.append(buildings_tmp)
    cluster_tmp = buildings[closest_pt_idx[i]: closest_pt_idx[i] + 1]
    cluster_list.append(cluster_tmp)

fig, ax = ox.plot_graph(G, bgcolor="w", node_size=1, node_color="gray", edge_color="#aaa", show=False, close=False)
for i in range(n_clusters):
    ax.scatter(cluster_list[i]['center'].x, cluster_list[i]['center'].y, c="#000000", marker="s", alpha=1, zorder=4)
ax.scatter(buildings['center'].x, buildings['center'].y, c=buildings['class'], alpha=1, zorder=3)

NameError: name 'model' is not defined

### 2. Path planning -- Intra-class
####  Finding the best route of each cluster

In [7]:
def TSP_Pandana(G_pan, cluster, buildings, print_msg=False):
    # Find nearest node in graph
    node_cluster = G_pan.get_node_ids([cluster['center'].x], [cluster['center'].y]).iloc[
        0]  # x,y in pandana must be pandas.series
    nodes_buildings = list(G_pan.get_node_ids(buildings['center'].x, buildings['center'].y))

    # For output the name of path
    node_name = {}  # mapping of node to building name
    node_name[node_cluster] = cluster['name']
    for i in range(len(nodes_buildings)):
        node_name[nodes_buildings[i]] = buildings.iloc[i]['name']

    # Init
    p = node_cluster
    # if print_msg:
    #     print(node_name[p], end='')
    travel_path = []
    residual_nodes_buildings = nodes_buildings.copy()
    total_distance = 0

    # TSP - Implement the Ｎearest Ｎeighbor based TSP　algorithm
    # use Pandana to find the shortest path
    # *************** your code begin ***************
    print_travel_path = []

    cur_node = node_cluster
    print_travel_path.append(node_name[cur_node])
    next_node = None
    while len(residual_nodes_buildings) != 0:
        cur_dis = float("inf")
        for i in range(len(residual_nodes_buildings)):
            path_length = G_pan.shortest_path_length(cur_node, residual_nodes_buildings[i])
            if path_length < cur_dis:
                cur_dis = path_length
                next_node = residual_nodes_buildings[i]
        travel_path.append(list(G_pan.shortest_path(cur_node, next_node)))
        cur_node = next_node
        residual_nodes_buildings.remove(next_node)
        print_travel_path.append(node_name[cur_node])
        total_distance += cur_dis

    # print result
    if print_msg:
        for i in range(len(print_travel_path)):
            print(print_travel_path[i] + "——>", end='')
        print(print_travel_path[0])  # back to the source node

        print('Total distance: %f' % total_distance + "m")

    # path = list(G_pan.shortest_path(p, node_school))
    # *************** your code end ***************
    return travel_path

2.1 Use greedy algorithm to solve TSP problem

In [8]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
edges = edges.reset_index()
G_pan = pandana.Network(nodes['x'], nodes['y'], edges['u'], edges['v'], edges[['length']], twoway=False)

fig, ax = ox.plot_graph(G, bgcolor="w", node_size=1, node_color="gray", edge_color="#aaa", show=False, close=False)
for i in range(n_clusters):
    ax.scatter(cluster_list[i]['center'].x, cluster_list[i]['center'].y, c="#000000", marker="s", alpha=1, zorder=4)
ax.scatter(buildings['center'].x, buildings['center'].y, c=buildings['class'], alpha=1, zorder=3)
for i in range(n_clusters):
    travel_path = TSP_Pandana(G_pan, cluster=cluster_list[i], buildings=buildings_list[i], print_msg=False)
    for path in travel_path:
        clear_output(wait=True)
        ox.plot_graph_route(G, path, ax=ax, orig_dest_size=0, route_alpha=0.5, route_colors='r', route_linewidths=2,
                            show=False, close=False)
        plt.pause(0.1)
        display(fig)

NameError: name 'ox' is not defined

As you can see from the above diagram, each center corresponds to buildings that form a closed-loop route,  
which can be used to adjust the location of the bookstore.

### 3. Path planning -- Inter-class
####  Finding the best routes of centers

3.1 Calculate distance matrix between centers

In [9]:
#澳门口岸中心 22.214919498514952, 113.54830707004824
start = G_pan.get_node_ids([113.54830707004824], [22.214919498514952]).iloc[0]
nearest_nodes = [start]
for i in range(5):
  node_building = G_pan.get_node_ids([center[i][0]],[center[i][1]]).iloc[0]
  nearest_nodes.append(node_building)

a = len(nearest_nodes)
distance_matrix = np.zeros(shape=(a,a))
for i in range(len(nearest_nodes)):
    for j in range(len(nearest_nodes)):
        distance_matrix[i][j] = G_pan.shortest_path_length(nearest_nodes[i], nearest_nodes[j])

NameError: name 'G_pan' is not defined

In [10]:
distance_matrix

NameError: name 'distance_matrix' is not defined

3.2 Find the optimal path by allocating the number of trucks, the capacity of each truck and the amount of goods demanded by each bookstore

In [11]:
data = {}
data['distance_matrix'] = distance_matrix
data['demands'] = [0,20,20,20,20,20]
data['vehicle_capacities'] = [60,60]
data['num_vehicles'] = 2
data['depot'] = 0

NameError: name 'distance_matrix' is not defined

In [12]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
 
 
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    #data = create_data_model()
 
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
 
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
 
 
    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
 
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
 
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
 
 
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
 
    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
 
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
 
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
 
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
 
 
if __name__ == '__main__':
    main()


NameError: name 'pywrapcp' is not defined

From the configuration results, it is clear that there are two trucks here, each with a loadable capacity of 60, and the amount of goods demanded by each bookstore is 20.  

Starting from the port of entry, the two trucks distribute the goods according to the routes of (0->1->3->2->0) and (0->5->4->0) respectively. The first truck is loaded with 60 capacity and send goods to the 1st, 3rd and 2nd bookstores in turn. The total distance of route is 26733m. The second truck is loaded with 40 capacity and send goods to the 5th and 4th bookstores in turn. The total distance of route is 4288m. 